# LIANA tumor vs normal core atlas v1

## 00. Libraries

In [4]:
import numpy as np
import pandas as pd
import scanpy as sc

#import plotnine as p9

#import liana as li
import decoupler as dc
#import omnipath as op

# Import DESeq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

In [5]:
adata = sc.read_h5ad(
    "/data/projects/2022/CRCA/results/v1/final/h5ads/paired_tumor-adata.h5ad"
)

In [6]:
sample_key = 'sample_id'
groupby = 'cell_type_fine'
condition_key = 'sample_type'

## 01. Pseudobulk

In [7]:
pdata = dc.get_pseudobulk(
    adata,
    sample_col=sample_key,
    groups_col=groupby,
    layer='counts',
    mode='sum',
    min_cells=10,
    min_counts=10000
)
pdata

AnnData object with n_obs × n_vars = 6907 × 28135
    obs: 'medical_condition', 'cancer_type', 'sample_id', 'sample_type', 'sample_tissue', 'anatomic_region', 'anatomic_location', 'tumor_stage', 'tumor_stage_TNM', 'tumor_stage_TNM_T', 'tumor_stage_TNM_N', 'tumor_stage_TNM_M', 'tumor_size', 'tumor_dimensions', 'tumor_grade', 'histological_type', 'microsatellite_status', 'mismatch_repair_deficiency_status', 'MLH1_promoter_methylation_status', 'MLH1_status', 'PIK3CA_status', 'SMAD4_status', 'NRAS_status', 'MSH6_status', 'FBXW7_status', 'NOTCH1_status', 'MSH2_status', 'PMS2_status', 'POLE_status', 'ERBB2_status', 'STK11_status', 'HER2_status', 'CTNNB1_status', 'BRAS_status', 'patient_id', 'sex', 'age', 'ethnicity', 'treatment_status_before_resection', 'treatment_drug', 'treatment_response', 'RECIST', 'matrix_type', 'tissue_dissociation', 'tissue_processing_lab', 'hospital_location', 'country', 'NCBI_BioProject_accession', 'SOLO_doublet_status', 'cell_type_coarse', 'cell_type_fine', 'psbulk

## 02. Subset the adata to remove the cell types that do not have both tumor and normal 

In [ ]:
# First, group the data by 'cell_type_fine' and check which cell types have both 'tumor' and 'normal'
valid_cell_types = []

# Iterate over unique cell types
for cell_type in adata.obs['cell_type_fine'].unique():
    # Subset the data for the current cell type
    subset = adata[adata.obs['cell_type_fine'] == cell_type]
    
    # Check if both 'tumor' and 'normal' are present
    if all(x in subset.obs['sample_type'].values for x in ['tumor', 'normal']):
        valid_cell_types.append(cell_type)

# Subset the AnnData object to keep only the valid cell types
adata_subset = adata[adata.obs['cell_type_fine'].isin(valid_cell_types)].copy()

## 03. PYDESEQ2

In [42]:
adata = adata_subset

In [49]:
dea_results = {}
quiet = True

for cell_group in pdata.obs[groupby].unique():
    try:
        # Select cell profiles
        ctdata = pdata[pdata.obs[groupby] == cell_group].copy()

        # Obtain genes that pass the edgeR-like thresholds
        # NOTE: QC thresholds might differ between cell types, consider applying them by cell type
        genes = dc.filter_by_expr(ctdata,
                                  group=condition_key,
                                  min_count=5, # a minimum number of counts in a number of samples
                                  min_total_count=10 # a minimum total number of reads across samples
                                  )

        # Filter by these genes
        ctdata = ctdata[:, genes].copy()

        # Build DESeq2 object
        # NOTE: this data is actually paired, so one could consider fitting the patient label as a confounder
        dds = DeseqDataSet(
            adata=ctdata,
            counts=pdata.X + 1,
            metadata=pdata.obs,
            design_factors="sample_type",
            ref_level=["sample_type", 'normal'], # set control as reference
            refit_cooks=True,
            quiet=quiet
        )

        # Compute LFCs
        dds.deseq2()
        # Contrast between stim and ctrl
        stat_res = DeseqStats(dds, contrast=["sample-type", 'tumor', 'normal'], quiet=quiet)
        stat_res.quiet = quiet
        # Compute Wald test
        stat_res.summary()
        # Shrink LFCs
        stat_res.lfc_shrink(coeff='sample-type_tumor_vs_normal') # {condition_key}_cond_vs_ref

        dea_results[cell_group] = stat_res.results_df

    except Exception as e:
        print(f"An error occurred for cell_type_fine '{cell_group}': {e}")
        continue


/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: adata was provided; ignoring counts.
  dds = DeseqDataSet(
/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: adata was provided; ignoring metadata.
  dds = DeseqDataSet(
/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: Same factor names in the design contain underscores ('_'). They will
                be converted to hyphens ('-').
  dds = DeseqDataSet(
/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/dds.py:716: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
Fitting dispersions...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.02 seconds.



Shrunk log2 fold change & Wald test p-value: sample-type tumor vs normal
         baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
A1BG    10.115952       -0.397364  0.129769 -3.494330  0.000475  0.002824
AAGAB    5.653157        0.307447  0.120828  3.014928  0.002570  0.011026
AAMP    10.228345        0.142425  0.073479  2.129223  0.033236  0.082092
AARS1    3.602263        0.404288  0.122620  3.702105  0.000214  0.001484
AARSD1   6.179178        0.526740  0.182165  3.463734  0.000533  0.003069
...           ...             ...       ...       ...       ...       ...
ZXDC     4.214497        0.176858  0.162822  1.619661  0.105305  0.198635
ZYG11B   4.173594        0.227674  0.129534  2.204152  0.027514  0.070634
ZYX      8.133982       -0.126846  0.127308 -1.312831  0.189240  0.306178
ZZEF1    7.972100        0.186147  0.120949  1.931891  0.053373  0.118138
ZZZ3     7.253166        0.078211  0.135322  0.793821  0.427300  0.556253

[6585 rows x 6 columns]


/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: adata was provided; ignoring counts.
  dds = DeseqDataSet(
/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: adata was provided; ignoring metadata.
  dds = DeseqDataSet(
/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: Same factor names in the design contain underscores ('_'). They will
                be converted to hyphens ('-').
  dds = DeseqDataSet(
/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/dds.py:716: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
Fitting dispersions...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.



Shrunk log2 fold change & Wald test p-value: sample-type tumor vs normal
          baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
AATF      4.193323        0.092994  0.099528  1.610583  0.107271  0.406204
ABCF1     6.055411        0.118523  0.189233  2.096653  0.036024  0.254859
ABHD14B  10.021402       -0.113541  0.143275 -1.837997  0.066063  0.338427
ABI1      6.827472        0.060646  0.093546  1.164917  0.244053  0.589294
ABI3      6.009743       -0.066815  0.122861 -1.349917  0.177043  0.523650
...            ...             ...       ...       ...       ...       ...
ZNF791    6.951296        0.023291  0.099741  0.517074  0.605104  0.852961
ZNF800    4.032408       -0.052592  0.105976 -1.055790  0.291064  0.637815
ZNHIT1    6.338959        0.024398  0.074259  0.506133  0.612763  0.858670
ZNHIT3    4.652724       -0.136912  0.112078 -2.050716  0.040295  0.271355
ZRANB2    9.219034        0.021414  0.088843  0.451157  0.651876  0.880016

[2357 rows x 6 columns]


/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: adata was provided; ignoring counts.
  dds = DeseqDataSet(
/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: adata was provided; ignoring metadata.
  dds = DeseqDataSet(
/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: Same factor names in the design contain underscores ('_'). They will
                be converted to hyphens ('-').
  dds = DeseqDataSet(
/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/dds.py:716: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
Fitting dispersions...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.



...

Shrunk log2 fold change & Wald test p-value: sample-type tumor vs normal
              baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
A1BG          6.884660       -0.623666  0.166156 -4.163943  0.000031  0.000297
A2M           9.704924       -0.127987  0.183907 -0.992698  0.320857  0.469675
AAAS          3.858986       -0.211445  0.164735 -1.642669  0.100451  0.198373
AAK1          6.904231        0.130290  0.135391  1.186681  0.235353  0.376758
AAMP          9.701489       -0.103882  0.098331 -1.181809  0.237281  0.379368
...                ...             ...       ...       ...       ...       ...
ZRANB2        9.927112       -0.060811  0.109779 -0.643497  0.519902  0.651251
ZSCAN16-AS1   4.637662       -0.068644  0.170462 -0.566506  0.571050  0.694664
ZSWIM6        7.674113        0.792275  0.225137  4.080986  0.000045  0.000402
ZSWIM7        9.368592       -0.472435  0.107767 -4.667455  0.000003  0.000044
ZYX          28.270835       -0.237573  0.149989 -1.926318

/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: adata was provided; ignoring counts.
  dds = DeseqDataSet(
/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: adata was provided; ignoring metadata.
  dds = DeseqDataSet(
/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: Same factor names in the design contain underscores ('_'). They will
                be converted to hyphens ('-').
  dds = DeseqDataSet(
/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/dds.py:716: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
Fitting dispersions...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.



Shrunk log2 fold change & Wald test p-value: sample-type tumor vs normal
          baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
AAK1      8.458190        0.129180  0.145960  1.180259  0.237897  0.375824
ABHD17A   5.516328       -0.270806  0.152324 -2.204625  0.027480  0.080332
ABI3      5.011926       -0.727598  0.181359 -4.456490  0.000008  0.000181
ABLIM1    4.262591        0.048729  0.183377  0.429862  0.667296  0.782611
ABRACL    8.742010       -0.229255  0.129397 -2.111716  0.034711  0.094948
...            ...             ...       ...       ...       ...       ...
ZNF207    6.482335        0.452844  0.134906  3.774793  0.000160  0.001846
ZNF292    5.127079        0.192800  0.176433  1.578412  0.114471  0.220172
ZNF331   15.417421       -0.006027  0.189746 -0.053296  0.957496  0.969913
ZNHIT1    4.311618       -0.006183  0.127766 -0.059619  0.952459  0.969913
ZRANB2    5.207061        0.049566  0.157800  0.447513  0.654505  0.772656

[1406 rows x 6 columns]


/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: adata was provided; ignoring counts.
  dds = DeseqDataSet(
/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: adata was provided; ignoring metadata.
  dds = DeseqDataSet(
/tmp/ipykernel_1028613/1180120716.py:22: UserWarning: Same factor names in the design contain underscores ('_'). They will
                be converted to hyphens ('-').
  dds = DeseqDataSet(
Fitting dispersions...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.02 seconds.

/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/home/kvalem/.conda/envs/crc_atlas/li

Shrunk log2 fold change & Wald test p-value: sample-type tumor vs normal
         baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
A1BG    10.487433    3.189644e-07  0.005136  0.179797  0.857312  0.996537
A2M      2.292853    1.222626e-07  0.005251  0.213739  0.830751  0.996537
AAAS     1.965413   -7.735946e-07  0.005376 -0.639369  0.522583  0.996537
AAK1     2.856669    1.147194e-06  0.006121  1.005784  0.314520  0.996537
AAMP     4.547064   -8.830468e-07  0.005413 -0.474306  0.635282  0.996537
...           ...             ...       ...       ...       ...       ...
ZSWIM7   3.031376   -1.263589e-06  0.005398 -0.843486  0.398957  0.996537
ZUP1     1.826346    2.251760e-08  0.005437  0.018055  0.985595  0.997666
ZYX      6.310446   -2.362966e-06  0.003995 -1.282551  0.199650  0.996537
ZZEF1    2.224566    7.403799e-07  0.005801  0.636330  0.524561  0.996537
ZZZ3     1.642247    1.054006e-06  0.006301  1.065069  0.286845  0.996537

[6284 rows x 6 columns]


## 04. Concatenate results across cell types

In [50]:
dea_df = pd.concat(dea_results)
dea_df = dea_df.reset_index().rename(columns={'level_0': groupby,'level_1':'index'}).set_index('index')
dea_df.head()

,cell_type_fine,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
index,,,,,,,
A1BG,B cell activated,10.115952,-0.397364,0.129769,-3.494330,0.000475,0.002824
AAGAB,B cell activated,5.653157,0.307447,0.120828,3.014928,0.002570,0.011026
AAMP,B cell activated,10.228345,0.142425,0.073479,2.129223,0.033236,0.082092
AARS1,B cell activated,3.602263,0.404288,0.122620,3.702105,0.000214,0.001484
AARSD1,B cell activated,6.179178,0.526740,0.182165,3.463734,0.000533,0.003069


## 05. Save Differential Expression results

In [55]:
dea_df.to_csv("dea.csv")

## 06. Subset the adata to those and normalize the counts

In [52]:
adata = adata[adata.obs[condition_key]=='tumor'].copy()

In [53]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

## 07. Combine the DEA results with the ligand-receptor interactions

In [ ]:
lr_res = li.multi.df_to_lr(adata,
                           dea_df=dea_df,
                           resource_name='consensus', # NOTE: uses HUMAN gene symbols!
                           expr_prop=0.1, # calculated for adata as passed - used to filter interactions
                           groupby=groupby,
                           stat_keys=['stat', 'pvalue', 'padj'],
                           use_raw=False,
                           complex_col='stat', # NOTE: we use the Wald Stat to deal with complexes
                           verbose=True,
                           return_all_lrs=False,
                           )

## 08. Visualize distribution of the interactions

In [ ]:
lr_res = lr_res.sort_values("interaction_stat", ascending=False)
lr_res['interaction_stat'].hist(bins=50)

## 09. Top interactions Wald statistic

In [ ]:
li.pl.tileplot(liana_res=lr_res,
               fill = 'expr',
               label='padj',
               label_fun = lambda x: '*' if x < 0.05 else np.nan,
               top_n=15,
               orderby = 'interaction_stat',
               orderby_ascending = False,
               orderby_absolute = False,
               source_title='Ligand',
               target_title='Receptor',
               )

## 10. Ligand-Receptor Plot

In [ ]:
plot = li.pl.dotplot(liana_res=lr_res,
                     colour='interaction_stat',
                     size='ligand_pvalue',
                     inverse_size=True,
                     orderby='interaction_stat',
                     orderby_ascending=False,
                     orderby_absolute=True,
                     top_n=10,
                     size_range=(0.5, 4)
                     )

# customize plot
(
    plot
    + p9.theme_bw(base_size=14)
    # fill cmap blue to red, with 0 the middle
    + p9.scale_color_cmap('RdBu_r', midpoint=0, limits=(-10, 10))
    # rotate x
    + p9.theme(axis_text_x=p9.element_text(angle=90), figure_size=(11, 6))

)